In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
# 获取数据
df = pd.read_csv('豆瓣电影数据.csv',engine = 'python')

print('数据条数有%i条'%len(df))
print('数据字段为：',df.columns.to_list)

数据条数有2876条
数据字段为： <bound method IndexOpsMixin.tolist of Index(['Unnamed: 0', 'imdb_id', 'imdb_n_comment', 'imdb_score', 'name',
       'rating_people', 'score', '上映日期', '主演', '制片国家/地区', '又名', '导演', '片长',
       '类型', '编剧', '语言', '主演人数', 'r5', 'r4', 'r3', 'r2', 'r1'],
      dtype='object')>


In [14]:
# 提取需要清洗的数据
data = df[['name','导演','主演']]
data.head()

,name,导演,主演
0,团圆,王全安,卢燕 / 凌峰 / 徐才根 / 马晓晴 / 莫小棋
1,非诚勿扰2,冯小刚,葛优 / 舒淇 / 孙红雷 / 姚晨 / 安以轩 / 邵兵 / 廖凡 / 邬逸聪 / 关晓...
2,无蝉的夏天,苗月,吕聿来 / 田璐菡
3,狄仁杰之通天帝国,徐克,刘德华 / 刘嘉玲 / 李冰冰 / 梁家辉 / 邓超 / 泰迪·罗宾 / 吴耀汉 / 姚橹...
4,我是植物人,王竞,李乃文 / 冯波


In [31]:
data_dy = data['导演'].str.split('/',expand = True)
data_zy = data['主演'].str.split('/',expand = True)
data_dy.columns = ['dy' + str(i) for i in range(len(data_dy.columns))]
data_zy.columns = ['zy' + str(i) for i in range(len(data_zy.columns))]
data_dy,data_zy

(                dy0   dy1   dy2   dy3   dy4   dy5   dy6   dy7   dy8   dy9  \
 0               王全安  None  None  None  None  None  None  None  None  None   
 1               冯小刚  None  None  None  None  None  None  None  None  None   
 2                苗月  None  None  None  None  None  None  None  None  None   
 3                徐克  None  None  None  None  None  None  None  None  None   
 4                王竞  None  None  None  None  None  None  None  None  None   
 5               松太加  None  None  None  None  None  None  None  None  None   
 6               乌尔善  None  None  None  None  None  None  None  None  None   
 7                路阳  None  None  None  None  None  None  None  None  None   
 8                沈东  None  None  None  None  None  None  None  None  None   
 9          西尔扎提·亚合甫  None  None  None  None  None  None  None  None  None   
 10              卢卫国  None  None  None  None  None  None  None  None  None   
 11               李珞  None  None  None  None  None  None  None  

In [39]:
# 将导演、主演和剧名合并为新的数据表
data_new = data_dy.join(data_zy).join(data['name'])
data_new.head()

,dy0,dy1,dy2,dy3,dy4,dy5,dy6,dy7,dy8,dy9,...,zy56,zy57,zy58,zy59,zy60,zy61,zy62,zy63,zy64,name
0,王全安,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,团圆
1,冯小刚,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,非诚勿扰2
2,苗月,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,无蝉的夏天
3,徐克,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,狄仁杰之通天帝国
4,王竞,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,我是植物人


In [57]:
data_re = pd.DataFrame(columns=['name','导演','主演'])

col_dy = data_dy.columns
col_zy = data_zy.columns

for dy in col_dy:
    for zy in col_zy:
        data_i = data_new[['name',dy,zy]].dropna()
        data_i.columns = ['name','导演','主演']
        data_re = pd.concat([data_re,data_i])
data_re.head()

,name,导演,主演
0,团圆,王全安,卢燕
1,非诚勿扰2,冯小刚,葛优
2,无蝉的夏天,苗月,吕聿来
3,狄仁杰之通天帝国,徐克,刘德华
4,我是植物人,王竞,李乃文


In [64]:
result = data_re.groupby(['导演','主演']).count()
result.reset_index(inplace=True)
result.columns = ['导演','主演','合作次数']
result.head()

,导演,主演,合作次数
0,(小)徐克,刘彬,1
1,(小)徐克,叶露,1
2,(小)徐克,姜玉玲,2
3,(小)徐克,孟祥龙,1
4,(小)徐克,张安琪,3


In [66]:
writer = pd.ExcelWriter('output.xlsx')
result.to_excel(writer,'sheet1')
writer.save()